In [ ]:
## Load any changes to local modules
%load_ext autoreload
%autoreload 2

import os
import sys

pwd = %pwd
project_dir = '{0}/../../../'.format(pwd)
module_path = os.path.abspath(os.path.join(project_dir))
if module_path not in sys.path:
    sys.path.append(module_path)

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
import re
import numpy as np
%matplotlib inline
from IPython.display import display

import sqlalchemy
import sqlite3

from nbcpact import AnalyzeQuantCompare,Peptide,PeptideGroup,UcbreUtils,PeptidesFromPeptideListBuilder


In [ ]:
file_path = "../../../test/data/peptideList.csv"
peptide_generator = PeptidesFromPeptideListBuilder(peptide_list_file=file_path)
peptides = peptide_generator.generate_peptides()
pep = peptides[0]

print("ip2_peptide={0}, sequence={1}, mod_locs={2}, ptm_indices={3}, area_ratio={4}".format(pep.ip2_peptide, pep.sequence, pep.mod_locs, pep.ptm_indices, pep.area_ratio))
print("area_ratios={0}, annotation={1}, uniprot_ids={2}, run_counter={3}".format(pep.area_ratios, pep.annotation, pep.uniprot_ids, pep.run_counter))
print("decoy={0},unique1={1}".format(pep.decoy, pep.unique1))


"""
ip2_peptide=None, sequence=None, mod_locs=None, ptm_indices=None, area_ratio=None,
                 area_ratios=None, annotation=None, uniprot_ids=None, run_counter=None, decoy=None,
                 unique1=None
"""

In [ ]:
from enum import Enum

class DataType(Enum):
    Float = 1

class PeptideFromPD2_1Generator:
    
    __psm_cys_mod_pattern = re.compile(r'C(\d+)\(isoTO\w*\)')
    __exp_pattern = r'(KEA_EN80\d)_\d.raw'
    __peptide_mod_pattern = re.compile(r'.*\d.isoTOP TEV [heavyLight]+ \[(.+)\].*')
    __peptide_cys_mod_pattern = re.compile(r'C(\d+)')
    
    __data_cache = {}

    
    def __init__(self, pd_study_dir="/usca/asperapoc/NB-CPACT/NB-CPACT-NIBR/NIBR_loaded_UCB_EN80/Analysis/PD2.1/"):
        self.__pd_study_dir = pd_study_dir
        # TODO: Look for the latest pdResult file
        file_path = '{0}/{1}'.format(pd_study_dir, 'KEA_isoTOP_DN_All.pdResult')
        self.__connection = sqlite3.connect(file_path)
        self.__target_psms = self.get_target_psms()
        modifications = self.get_found_modifications()
        
        value = float(modifications[modifications['Name'] == 'IsoTOP heavy']['DeltaMonoisotopicMass'])
        self.__isoTopHeavyMass = "{0:.2f}".format(value)
        value = float(modifications[modifications['Name'] == 'IsoTOP light']['DeltaMonoisotopicMass'])
        self.__isoTopLightMass = "{0:.2f}".format(value)
        
    
    def __process_psm_modifications(self, modifications):
        mod_locs = list(map(int, re.findall(self.__psm_cys_mod_pattern, modifications)))
        return mod_locs
    
    def get_found_modifications(self):
        sqlStr = "SELECT * FROM FoundModifications"
        df = pd.read_sql(sqlStr, self.__connection)
        return df
    
    def get_target_psms(self):
        sqlStr = """
        SELECT
        Sequence,
        ModifiedSequence,
        Modifications,
        ParentProteinAccessions,
        ParentProteinDescriptions,
        SpectrumFileName,
        QuanChannel,
        QuanValueIsoTOPLight,
        QuanValueIsoTOPHeavy
        FROM TargetPsms
        WHERE (QuanChannelID = 1 OR QuanChannelID = 2)
        """
        df = pd.read_sql(sqlStr, self.__connection)
        
        df['mod_locs'] = df.Modifications.apply(self.__process_psm_modifications)
        df['str_mod_locs'] = df['mod_locs'].astype(str)
        df['EXPERIMENT'] = df.SpectrumFileName.str.extract(self.__exp_pattern)
        
        return df
    
    def __process_modifications(self, modifications):
        match = self.__peptide_mod_pattern.match(modifications)
        if match:
            mod_locs = list(map(int, re.findall(self.__peptide_cys_mod_pattern, match.group(1))))
            return mod_locs
        
    
    def __create_run_counter(self, row, experiments=['KEA_EN801', 'KEA_EN802', 'KEA_EN803']):
        sequence = row['Sequence']
        mod_locs = str(row['str_mod_locs'])
        targetPsmsDF = self.__target_psms
        
        psms = targetPsmsDF[(targetPsmsDF.Sequence == sequence) & (targetPsmsDF['str_mod_locs'] == mod_locs)]
        psm_exps = list(psms['EXPERIMENT'])
        run_counter = [i in psms_exps for i in experiments]

        return str(run_counter)
    
    def __get_global_mod_position(self, row):
        data_cache_name = 'target_prot_sequences'
        
        if not data_cache_name in self.__data_cache.keys():
            sqlStr = """
                        SELECT
                        t1.PeptideGroupID,t3.Sequence
                        FROM TargetPeptideGroups t1,
                        TargetPeptideGroupsTargetProteins t2,
                        TargetProteins t3
                        WHERE t1.PeptideGroupID = t2.TargetPeptideGroupsPeptideGroupID
                        AND t2.TargetProteinsUniqueSequenceID = t3.UniqueSequenceID
                    """
            
            df = pd.read_sql(sqlStr, self.__connection)
            self.__data_cache[data_cache_name] = df
            
        peptide_sequence = row['Sequence']
        
        local_positions = [int(numeric_string) for numeric_string in  row['mod_locs']]
        peptideGroupID = row['PeptideGroupID']
        
        df = self.__data_cache[data_cache_name]
        proteinSequences = df[df['PeptideGroupID'] == peptideGroupID]['Sequence']
        
        peptide_starts = []
        for proteinSequence in proteinSequences.values:
            peptide_starts.append((proteinSequence.find(peptide_sequence)))
            
        global_positions = []
        for peptide_start in peptide_starts:      
            pos_strings = []

            for local_position in local_positions:
                global_pos = np.nan if peptide_start == -1 else (local_position + peptide_start)
                pos_strings.append('C{0}'.format(global_pos))

            global_positions.append(','.join(pos_strings))
            
        
        return global_positions   
    
    def __create_ip2peptide(self, row):

        modifications = row['Modifications_best_positions']
        sequence = row['Sequence']
        mod_locs = row['mod_locs']
        
        mass = modifications
        if 'isoTOP TEV heavy' in modifications:
            mass = self.__isoTopHeavyMass
        elif 'isoTOP TEV Light' in modifications:
            mass = self.__isoTopLightMass
            
        for mod_loc in mod_locs:
            sequence = '{0}C({1}){2}'.format(sequence[0:mod_loc-1], mass, sequence[mod_loc:])

        return sequence


    def __extract_values(self, binary_data, n=None, dataType=None):
        """
        values: bytes from the blob
        n: number of channels
        t: type of data
            'decimal' for values in decimal format such as Abundances
            'integer_number' for values such as 'Found in'

        """
        result = []

        if dataType == DataType.Float:
            for i in range(n):
                sub = binary_data[9*i:9*i+8]
                result.append(struct.unpack("d",sub)[0])
        else:
            return binary_data

        if len(result) == 1:
            return result[0]
        else:
            return result
    
    
    def get_target_peptides_from_pdresult(self):
        sqlStr = """
            SELECT
            PeptideGroupID, 
            Checked,
            Confidence,
            Sequence,
            Modifications_all_positions,
            Modifications_best_positions,
            Contaminant,
            QvalityPEP,
            Qvalityqvalue,
            ParentProteinGroupCount,
            ParentProteinCount,
            PsmCount,
            MasterProteinAccessions,
            MissedCleavages,
            TheoreticalMass,
            QuanInfo,
            IonsScoreMascot,
            ConfidenceMascot,
            PercolatorqValueMascot,
            PercolatorPEPMascot,
            AbundanceRatios,
            Abundances,
            FoundinSamples
            FROM TargetPeptideGroups tpg
            WHERE AbundanceRatios IS NOT NULL LIMIT 10;
            """
        
        df = pd.read_sql(sqlStr, self.__connection)
        
        df['mod_locs'] = df['Modifications_best_positions'].apply(self.__process_modifications)
        
        df['ip2_peptide'] = df.apply(self.__create_ip2peptide, axis=1)
        
        df['str_mod_locs'] = df['mod_locs'].astype(str) # To help with merge
        df['run_counter'] = df.apply(self.__create_run_counter, axis=1)       
        
        df['AbundanceRatios'] = df['AbundanceRatios'].apply(self.__extract_values, n=1, dataType=DataType.Float)
        df['Log2Ratio'] = df['AbundanceRatios'].apply(np.log2)
        
        df['ptm_indices'] = df.apply(self.__get_global_mod_position, axis=1)
        
        df = df.drop(['Abundances', 'FoundinSamples'], axis=1)
        
        return df
    
    def get_target_peptides(self):
        # TODO: Look for any xlsx file
        file_path = "{0}/KEA_isoTOP_DN_High_Med_All.xlsx".format(self.__pd_study_dir)
        df = pd.read_excel(file_path)
        ## Clean up dataframe
        df = df[~df['Abundance Ratio (log2): (IsoTOP Light) / (IsoTOP Heavy)'].isnull()]
        df['mod_locs'] = df['Modifications'].apply(self.__process_modifications)
        df['run_counter'] = df.apply(self.__create_run_counter, axis=1)
        
        return df

pep_generator = PeptideFromPD2_1Generator()



## 

In [ ]:
peptide_from_pdresultsDF = pep_generator.get_target_peptides_from_pdresult()
peptide_from_pdresultsDF.T

In [ ]:
sequence = 'SDFCSDFSDCSDFSDFSDF'

mod_loc=4
mass = '123.00'

sequence[0:mod_loc-1] + 'C' + mass + sequence[mod_loc:]

In [ ]:
data = {}

## Transfer some values directly
data['uniprot_ids'] = peptide_from_pdresultsDF['MasterProteinAccessions']
data['area_ratio'] = peptide_from_pdresultsDF['AbundanceRatios']
data['sequence'] = peptide_from_pdresultsDF['Sequence']



# Do later
## TODO: Global PTM
data['ptm_indices'] = peptide_from_pdresultsDF['ptm_indices']
## Detailed protein annot
data['annotation'] = None
## The IP2 semicolon array that RunCounter is made from
data['unique1'] = None

data['run_counter'] = peptide_from_pdresultsDF['run_counter']
## Decoy is always false as the SQL Takes care of that. 
data['decoy'] = False
## TODO: Create ip2_peptide
ip2_peptide=peptide_from_pdresultsDF['ip2_peptide']

df = pd.DataFrame(data=data)


In [ ]:
def __init_peptide(self, row):
    peptide = Peptide(sequence=row['sequence'],
                      mod_locs=row['mod_locs'],
                      ptm_indices=row['ptm_indices'],
                      area_ratio=row['area_ratio'],
                      area_ratios=row['area_ratios'],
                      annotation=row['annotation'],
                      uniprot_ids=row['uniprot_ids'],
                      run_counter=row['run_counter'],
                      decoy=row['decoy'],
                      unique1=row['UNIQUE_1'],
                      ip2_peptide=row['ip2_peptide'])

        return peptide

df.apply(__init_peptide)

In [ ]:
pepgroup_df[pepgroup_df.Sequence == 'CGEEIAVQFVDMVK']

In [ ]:
x = b'\x06\x00\x00\x00c@sA\x01\xfe\xff\xffG\x8b\x7feA\x01'

from enum import Enum

class DataType(Enum):
    Float = 1

def extract_values(binary_data, n=None, dataType=None):
    """
    values: bytes from the blob
    n: number of channels
    t: type of data
        'decimal' for values in decimal format such as Abundances
        'integer_number' for values such as 'Found in'

    """
    result = []
    
    if dataType == DataType.Float:
        for i in range(n):
            sub = binary_data[9*i:9*i+8]
            result.append(struct.unpack("d",sub)[0])
    else:
        return binary_data

    if len(result) == 1:
        return result[0]
    else:
        return result
    
file_path = "/usca/asperapoc/NB-CPACT/NB-CPACT-NIBR/NIBR_loaded_UCB_EN80/Analysis/PD2.1/KEA_isoTOP_DN_All.pdResult"
pd_connection = sqlite3.connect(file_path)

sqlStr = """
            SELECT Sequence, Abundances, AbundanceRatios, FoundInSamples FROM TargetPeptideGroups tpg 
            WHERE AbundanceRatios IS NOT NULL AND Sequence = 'CGEEIAVQFVDMVK'
        """
targetPeptideGroupsDF = pd.read_sql(sqlStr, pd_connection)
print(targetPeptideGroupsDF.AbundanceRatios.apply(extract_values, n=1, dataType=DataType.Float).apply(np.log2))
print(targetPeptideGroupsDF.Abundances.apply(extract_values, n=2, dataType=DataType.Float))
print(targetPeptideGroupsDF.FoundinSamples.apply(extract_values, n=1))

In [ ]:
peptide_df[peptide_df.Sequence == 'CGEEIAVQFVDMVK'].T

In [ ]:
def unpack_type(barray, n, t):
    """
    extract BLOB from pdResult file
    barray: barray that holds blob
    n: number of channels
    t: type of data
        'decimal' for values in decimal format such as Abundances
        'integer_number' for values such as 'Found in'

    """
    
    tmp = []
    if barray:
        if t == 'decimal':
            for i in range(n):
                tmp.append(struct.unpack("d",barray[9*i:9*i+8]))
        if ((t == 'integer_number') or (t == 'integer_text')):
            for i in range(n):
                tmp.append(struct.unpack("i",df[5*i:5*i+4]))
    # take care of the missing values     
    else: 
        if t == 'decimal':
            tmp = [0.0]*n    
        if ((t == 'integer_number') or (t == 'integer_text')):
            tmp = [0]*n

    return tmp


barray = b'{\x14\xaeG\xe1z\x84?\x01'
unpack_type(barray, 5, 'decimal')